# Training Pipeline

# Imports
## Pip Packages

In [1]:
import os

if not os.path.exists('data'):
    new_directory_path = "..\\..\\"
    os.chdir(new_directory_path)

from torch.utils.data import Dataset
import numpy as np

## My Modules

In [2]:
from src.datasets import CocoFreeView
from src.simulation import process_scanpaths, downsample
from src.noise import add_random_center_correlated_radial_noise

# Code
## Data

In [ ]:
# TODO Compute image embeddings just once

class PathCocoFreeViewDataset(Dataset):
    '''
    The noisy and downsampled simulated eye-tracking and the section of the scanpath that fits entirely in that part
    '''

    def __init__(self, sample_size=-1, sampling_rate=60, downsample=200):
        super().__init__()
        self.data = CocoFreeView()
        self.sampling_rate = sampling_rate
        # more than 90% generate at least 20 samples after downsampling to 200 samples per second
        self.sample_size = sample_size
        self.downsample = downsample

    def __len__(self):
        return len(self.data)
    
    @staticmethod
    def __extract_random_period(size, noisy_samples, fixations, fixation_mask):
        idx = int(np.random.randint(0, noisy_samples.shape[1] - size + 1, 1))
        last_idx = idx + size - 1
        start_fixation = -1
        # get the fisrt fixation and if it is not completely included get the next one
        if fixation_mask[idx] > 0:
            if (idx - 1) >= 0 and fixation_mask[idx - 1] == fixation_mask[idx]:
                start_fixation = fixation_mask[idx] + 1
            else:
                start_fixation = fixation_mask[idx]
        else:
            # if the first value is a saccade look for the first fixation
            current_idx = idx
            while current_idx < (idx + size) and fixation_mask[current_idx] == 0:
                current_idx += 1
            if current_idx == (idx + size):
                # if there is not a fixation return an empty array
                return noisy_samples[:, idx:idx + size],np.array([])
            else:
                start_fixation = fixation_mask[current_idx]
        # search the last fixation
        if fixation_mask[last_idx] > 0:
            if (last_idx + 1) < fixation_mask.shape[0] and fixation_mask[last_idx + 1] == fixation_mask[last_idx]:
                end_fixation = fixation_mask[last_idx] - 1
            else:
                end_fixation = fixation_mask[last_idx]
        else:
            current_idx = last_idx
            while current_idx > idx and fixation_mask[current_idx] == 0:
                current_idx -= 1
            if current_idx == idx:
                end_fixation = start_fixation
            else: 
                end_fixation = fixation_mask[current_idx]
        # the mask are saved shifted in order to assign 0 to the saccade samples
        start_fixation -= 1
        end_fixation -= 1
        x = noisy_samples[:, idx:idx + size]
        y = fixations[:, start_fixation: end_fixation + 1]
        return x, y

    def __getitem__(self, index):
        # TODO save the simulated scanpath without noise due that they can be reused
        gaze_list, fixations, fixation_mask = process_scanpaths(self.data,
                                                                index, self.sampling_rate,
                                                                get_scanpath=True,
                                                                get_fixation_mask=True)
        down_gaze = downsample(gaze_list, down_time_step=self.downsample)
        noisy_gaze, _ = add_random_center_correlated_radial_noise(down_gaze, [320//2, 512//2], 1/16,
                                                                  radial_corr=.2,
                                                                  radial_avg_norm=4.13,
                                                                  radial_std=3.5,
                                                                  center_noise_std=100,
                                                                  center_corr=.3,
                                                                  center_delta_norm=300,
                                                                  center_delta_r=.3)

        if self.sample_size != -1:
            x_batch, y_batch = [], []
            for i in range(len(fixations)):
                random_gaze, random_fixations = self.__extract_random_period(self.sample_size,
                                                                             noisy_gaze[i],
                                                                             fixations[i],
                                                                             fixation_mask[i])
                x_batch.append(random_gaze)
                y_batch.append(random_fixations)
        return x_batch, y_batch

# Code
## Data

In [17]:
dataset = PathCocoFreeViewDataset(sample_size=4)


In [14]:
len(dataset)

43048

In [21]:
idx = np.random.randint(0,len(dataset), 10, dtype=int)
dataset[idx]

(2, 10)


C:\Users\ulloa\AppData\Local\Temp\ipykernel_14496\1570080967.py:22: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  idx = int(idx)


UnboundLocalError: cannot access local variable 'end_fixation' where it is not associated with a value